## queryset对象
    可进行切片操作

### filter
    返回符合条件的模型组成的queryset
    
    
### exclude
    与filter相反，其会排除掉符合条件的对象
    
    
### get
    返回符合条件的单个模型对象，匹配到多个或没有匹配均会引发异常
    
    
    
### create
    实例化一个模型对象，并保存到数据库，调用此方法后无需再手动调用save()方法


### get_or create
    尝试获取一个对象，若不存在，则直接创建并保存到数据库中
    其返回值是一个二元组，0为模型对象，1为是否新创建对象True/False
    
    
### bulk_create
    一次性创建多个对象

In [ ]:
# 假设Book模型仅含有一个name这个字段
Book.objects.bulk_create(
    Book(name='三国')，
    Book(name='水浒')，
)

### values
    model.objects.values(field, field..)
    返回值仍为queryset, 但是其中存储的不再是模型实例对象，而是包含所选字段的字典。如：
        ({'name': 'xiao', 'age':13}, {'name': 'jack', 'age': 12}....)
    如果需要提取的字段为关联模型的字段，则可通过'__'形式进行关联查找，如：
        Book.objects.values('book__author')
    倘若不提供field参数，则默认会将模型上所有的字段信息提取出来，并以字典形式返回。


### values_list
    类似values, 但是返回的queryset是由字段值组成的元组，因此不包含字段名。
    
    
### select_related
    用于优化数据库查询。
    在一对一，一对多关系中，一般会先查询出当前模型的数据，然后通过外键关系引用关联数据。由于第一次查询时，并未获取关联模型，因此引用外键数据时，需要再对数据库进行查询。
    使用该字段可以在第一次查询时将关联模型一并获取，进而避免后续的数据库查询操作。
#### 注意
    该字段只能用于确定唯一的外键关系中，因此无法用于多对多关系的查询，这一点从原理上很好理解，假设有多个关联对象，则无法确定关联目标。
    多对多关系的一次性查询可用prefetch_related

In [ ]:
# 获取模型数据
books = Book.objects.all()
for book in books:
    # 对作者名称的访问会再次引发查询
    print(book.author.name)
    

# 使用select_related字段
books = book.objects.select_related('author')
for book in books:
    # author已在内存中，不会再进行查询
    print(book.author.name)

### prefetch_related
    类似select_related，但用于多对多关系的快速获取。此时会将多对多关系中的所有关联数据获取出来。如：
        books = book.objects.select_related('tag_set')
        
    在多对多关系中，我们可能会想对查询的数据集进行进一步过滤。倘若在进行上面的快捷查询后，对结果调用filter等方法，则会重新进行数据库查询。
    为应对上述情况，则需要借助django.db.models中的Preftch类。

In [ ]:
books = book.objects.select_related('tag_set')
for book in books:
    # 这会使得前面的tag_set数据被弃用
    book.tag_set.filter(name__in=['励志', '财经'])
    
from django.db.models import Prefetch

# 相当于将过滤后的queryset作为tag_set的查询整体
prefetch = Prefetch('tag_set', queryset=Tag.objects.filter(name__in=['励志', '财经']))
books = book.objects.select_related(prefetch)

### defer
    查询时过滤掉不需要的字段。其返回的仍然是模型，而不是像values一样返回字典。如过滤掉书籍的内容：
        books.objects.defer('content')
    倘若后面访问被过滤的字段，则会重新查询数据库。
    
### only
    于defer相反，其仅查询only指定的字段。若访问未指定的字段，则会重新查询数据库。
    
    
    
### count
    计算查询集中数据的数量，该操作在数据库中直接执行，无需将数据先保存到python内存中，因此比len方法更高效
        Book.objects.filter().count()
        
  
### first
    返回queryset中的第一个对象
    

### last
    返回queryset中的最后一个对象
    
    
### aggregate
    使用聚合函数
    
    
### exists
    用于判断符合某个条件的数据是否存在。该方法比通过count或者queryset判断拥有更高的效率。
    Book.objects.filter(title__contains='如何').exists()

### distinct
    删除重复的数据
    Book.objects.filter(bookorder__price__gt=80).distinct()
    Book和bookorder是一对多关系，由于同一本书极可能存在多张订单，因此查询结果会包含同一本书的多个重复对象。此时可以清除重复对象。
    
    
    
### update
    更新符合条件的指定字段
    Book.objects.filter(price__lt=50).update(price=F('price')+10)
    价格低于50元的图书全部在原价上涨价10元。F表达式实现在各自原有价格涨价，而无需遍历过滤结果再进行价格修改。
    
    
### delete
    删除选定的对象，使用该方法式必须慎重，因为根据on_delete属性的设置情况不同，可能会引发关联数据的删除。
   